# 範例 19：完整的 Fine-Tuning 評估系統

包含多種評估指標和完整報告的評估系統！

## 學習目標
- 建立完整的評估流程
- 了解多維度評估方法
- 學會生成評估報告

## 前置需求
- LM Studio 運行中，Local Server 已啟動
- 安裝 openai 套件：`pip install openai`

## Step 1: 匯入套件並設定

In [ ]:
from openai import OpenAI
import json

client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="not-needed"
)

print("已連接到 LM Studio！")

## Step 2: 建立完整的模型評估器類別

In [ ]:
class ModelEvaluator:
    """
    模型評估器
    用於評估 Fine-Tuning 的效果
    """

    def __init__(self, base_model, finetuned_model=None):
        """
        初始化評估器

        參數：
            base_model: 原始模型名稱
            finetuned_model: 微調後模型名稱（可選）
        """
        self.base_model = base_model
        self.finetuned_model = finetuned_model
        self.results = []
        self.client = client

    def get_response(self, model, question, system_prompt=None):
        """取得模型回應"""
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        messages.append({"role": "user", "content": question})

        response = self.client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content

    def score_response(self, question, response, criteria):
        """
        使用 AI 評分回答品質（多維度）
        """
        prompt = f"""請評估以下回答的品質，針對每個標準給予 1-5 分。

問題：{question}

回答：{response}

評估標準：
{criteria}

請用以下 JSON 格式回答（只輸出 JSON）：
{{
    "accuracy": <1-5>,
    "completeness": <1-5>,
    "clarity": <1-5>,
    "usefulness": <1-5>,
    "overall": <1-5>,
    "comment": "簡短評語（一句話）"
}}"""

        eval_response = self.client.chat.completions.create(
            model=self.base_model,
            messages=[{"role": "user", "content": prompt}]
        )

        try:
            result = eval_response.choices[0].message.content
            start = result.find('{')
            end = result.rfind('}') + 1
            if start != -1 and end != 0:
                return json.loads(result[start:end])
        except:
            pass
        
        return {"overall": 0, "comment": "評估失敗"}

    def evaluate_single(self, question, system_prompt=None):
        """
        評估單一問題
        """
        criteria = """
        - accuracy（準確性）：資訊是否正確
        - completeness（完整性）：是否涵蓋所有重點
        - clarity（清晰度）：是否容易理解
        - usefulness（實用性）：是否有幫助
        - overall（整體）：綜合評分
        """

        result = {"question": question}

        # 評估原始模型
        base_response = self.get_response(
            self.base_model, question, system_prompt
        )
        base_score = self.score_response(question, base_response, criteria)
        result["base"] = {
            "response": base_response,
            "scores": base_score
        }

        # 如果有微調模型，也進行評估
        if self.finetuned_model:
            ft_response = self.get_response(
                self.finetuned_model, question, system_prompt
            )
            ft_score = self.score_response(question, ft_response, criteria)
            result["finetuned"] = {
                "response": ft_response,
                "scores": ft_score
            }

        self.results.append(result)
        return result

    def evaluate_batch(self, questions, system_prompt=None):
        """
        批次評估多個問題
        """
        print(f"開始評估 {len(questions)} 個問題...\n")

        for i, q in enumerate(questions, 1):
            print(f"[{i}/{len(questions)}] 評估中: {q[:30]}...")
            self.evaluate_single(q, system_prompt=system_prompt)

        return self.get_summary()

    def get_summary(self):
        """
        取得評估摘要
        """
        if not self.results:
            return "尚無評估結果"

        summary = {
            "total_questions": len(self.results),
            "base_model": {
                "avg_overall": 0,
                "avg_accuracy": 0,
                "avg_clarity": 0,
                "avg_completeness": 0,
                "avg_usefulness": 0
            }
        }

        # 計算原始模型平均分數
        base_scores = [r["base"]["scores"] for r in self.results]
        for metric in ["overall", "accuracy", "clarity", "completeness", "usefulness"]:
            values = [s.get(metric, 0) for s in base_scores if s.get(metric, 0) > 0]
            if values:
                summary["base_model"][f"avg_{metric}"] = sum(values) / len(values)

        # 如果有微調模型的結果
        if self.finetuned_model and "finetuned" in self.results[0]:
            summary["finetuned_model"] = {
                "avg_overall": 0,
                "avg_accuracy": 0,
                "avg_clarity": 0,
                "avg_completeness": 0,
                "avg_usefulness": 0
            }
            ft_scores = [r["finetuned"]["scores"] for r in self.results]
            for metric in ["overall", "accuracy", "clarity", "completeness", "usefulness"]:
                values = [s.get(metric, 0) for s in ft_scores if s.get(metric, 0) > 0]
                if values:
                    summary["finetuned_model"][f"avg_{metric}"] = sum(values) / len(values)

            # 計算改善幅度
            summary["improvement"] = {}
            for metric in ["overall", "accuracy", "clarity"]:
                base_val = summary["base_model"][f"avg_{metric}"]
                ft_val = summary["finetuned_model"][f"avg_{metric}"]
                summary["improvement"][metric] = ft_val - base_val

        return summary

    def print_report(self):
        """
        印出評估報告
        """
        summary = self.get_summary()

        print("\n" + "=" * 60)
        print("          Fine-Tuning 評估報告")
        print("=" * 60)

        print(f"\n評估問題數：{summary['total_questions']}")

        print(f"\n┌{'─'*58}┐")
        print(f"│ 原始模型 ({self.base_model})")
        print(f"├{'─'*58}┤")
        base = summary['base_model']
        print(f"│   整體評分：{base['avg_overall']:.2f}/5")
        print(f"│   準確性：  {base['avg_accuracy']:.2f}/5")
        print(f"│   完整性：  {base['avg_completeness']:.2f}/5")
        print(f"│   清晰度：  {base['avg_clarity']:.2f}/5")
        print(f"│   實用性：  {base['avg_usefulness']:.2f}/5")
        print(f"└{'─'*58}┘")

        if "finetuned_model" in summary:
            print(f"\n┌{'─'*58}┐")
            print(f"│ 微調模型 ({self.finetuned_model})")
            print(f"├{'─'*58}┤")
            ft = summary['finetuned_model']
            print(f"│   整體評分：{ft['avg_overall']:.2f}/5")
            print(f"│   準確性：  {ft['avg_accuracy']:.2f}/5")
            print(f"│   完整性：  {ft['avg_completeness']:.2f}/5")
            print(f"│   清晰度：  {ft['avg_clarity']:.2f}/5")
            print(f"│   實用性：  {ft['avg_usefulness']:.2f}/5")
            print(f"└{'─'*58}┘")

            print(f"\n┌{'─'*58}┐")
            print(f"│ 改善幅度")
            print(f"├{'─'*58}┤")
            imp = summary["improvement"]
            for metric, value in imp.items():
                arrow = "↑" if value > 0 else "↓" if value < 0 else "→"
                print(f"│   {metric}：{arrow} {value:+.2f}")
            print(f"└{'─'*58}┘")

        print("\n" + "=" * 60)

## Step 3: 建立評估器並準備測試問題

In [ ]:
# 建立評估器
evaluator = ModelEvaluator(
    base_model="gpt-oss-120b",
    # finetuned_model="my-finetuned-model"  # 如果有微調模型
)

# 測試問題集
test_questions = [
    "什麼是變數？請用簡單的方式解釋。",
    "Python 中 list 和 dictionary 有什麼差別？",
    "如何處理程式中的錯誤？",
    "什麼是遞迴？請舉例說明。",
    "解釋什麼是 API，以及為什麼要用它。"
]

print(f"準備評估 {len(test_questions)} 個問題")

## Step 4: 執行評估

In [ ]:
# 執行評估
system_prompt = "你是一位程式設計教師，用簡單的方式回答問題。"

evaluator.evaluate_batch(
    test_questions,
    system_prompt=system_prompt
)

## Step 5: 查看評估報告

In [ ]:
# 印出報告
evaluator.print_report()

## Step 6: 查看詳細結果

In [ ]:
# 查看每個問題的詳細評估
print("\n=== 詳細評估結果 ===")

for i, result in enumerate(evaluator.results, 1):
    print(f"\n{'─'*60}")
    print(f"問題 {i}：{result['question']}")
    print(f"{'─'*60}")
    
    base = result['base']
    scores = base['scores']
    
    print(f"\n回答：{base['response'][:150]}...")
    print(f"\n評分：")
    print(f"  - 整體：{scores.get('overall', 'N/A')}/5")
    print(f"  - 準確性：{scores.get('accuracy', 'N/A')}/5")
    print(f"  - 清晰度：{scores.get('clarity', 'N/A')}/5")
    print(f"  - 評語：{scores.get('comment', 'N/A')}")

## Step 7: 匯出評估結果

In [ ]:
def export_results(evaluator, filename):
    """
    匯出評估結果為 JSON 檔案
    """
    export_data = {
        "summary": evaluator.get_summary(),
        "details": evaluator.results
    }
    
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(export_data, f, ensure_ascii=False, indent=2)
    
    print(f"評估結果已匯出到 {filename}")

# 匯出結果
export_results(evaluator, "evaluation_results.json")

## 評估指標說明

### 五個評估維度

| 維度 | 英文 | 說明 |
|------|------|------|
| 準確性 | accuracy | 資訊是否正確無誤 |
| 完整性 | completeness | 是否涵蓋所有重點 |
| 清晰度 | clarity | 表達是否清楚易懂 |
| 實用性 | usefulness | 對使用者是否有幫助 |
| 整體 | overall | 綜合評分 |

### 評分標準

- **5 分**：優秀，完全符合標準
- **4 分**：良好，大致符合標準
- **3 分**：普通，基本符合標準
- **2 分**：不佳，有明顯問題
- **1 分**：很差，完全不符合

## 練習區

In [ ]:
# 建立你自己的評估

# 1. 準備你的測試問題
my_questions = [
    "你的問題 1",
    "你的問題 2",
]

# 2. 建立評估器
# my_evaluator = ModelEvaluator(base_model="gpt-oss-120b")

# 3. 執行評估
# my_evaluator.evaluate_batch(my_questions)

# 4. 查看報告
# my_evaluator.print_report()

## 恭喜完成所有範例！

### 你學到了：

1. **基礎對話**（範例 1-5）
   - 與 Ollama 進行對話
   - 多輪對話和串流輸出
   - 系統提示詞和角色扮演

2. **LM Studio 使用**（範例 6-11）
   - OpenAI 相容 API
   - 使用 OpenAI SDK
   - 通用聊天程式

3. **RAG 技術**（範例 12-14）
   - 簡易 RAG 系統
   - 向量搜尋
   - 文件問答

4. **Fine-Tuning 相關**（範例 15-19）
   - 準備訓練資料
   - 自訂 Ollama 模型
   - 資料增強
   - 模型評估

### 下一步建議：

- 嘗試用學到的技術建立自己的應用
- 探索更多進階功能
- 持續練習和實驗！